In [43]:
import numpy as np
import cv2
from tensorflow.keras import applications, models, layers
import os

In [47]:
def get_xy(path):
  x = []
  y = []
  dir_list = os.listdir(path)
  for i in range(len(dir_list)):
    dir_path = path + "/" + dir_list[i]
    dir_name = os.listdir(dir_path)

    for j in range(len(dir_name)):
      full_dir_path = dir_path + "/" + dir_name[j]

      img = cv2.imread(full_dir_path)
      img = cv2.resize(img, (32,32))
      x.append(img)
      y.append(i)
  return (np.array(x), np.array(y))

In [49]:
def get_data() :
  train_path ="./train data/"
  test_path ="./test data/"
  x_train, y_train = get_xy(train_path)
  x_test, y_test = get_xy(test_path)
  return (x_train, y_train), (x_test, y_test)

In [54]:
def make_model() :
  resnet50 = applications.resnet50.ResNet50(include_top=False, weights="imagenet", input_shape=(32,32,3))
  resnet50.trainable = False
  model = models.Sequential()
  model.add(resnet50)
  model.add(layers.Flatten())
  model.add(layers.Dense(1024, activation="relu"))
  
  # 분류 데이터가 몇종류인지
  model.add(layers.Dense(5, activation="softmax"))
  model.compile(loss = "sparse_categorical_crossentropy", optimizer="adam", metrics = ["accuracy"])
  print(model.summary())
  return model

In [57]:
def learning() :
  (x_train, y_train), (x_test, y_test) = get_data()
  model = make_model()
  log = model.fit(x_train, y_train, epochs=2, batch_size=16)
  score = model.evaluate(x_test, y_test)
  print(score)
learning()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 1, 1, 2048)        23587712  
_________________________________________________________________
flatten_2 (Flatten)          (None, 2048)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 1024)              2098176   
_________________________________________________________________
dense_5 (Dense)              (None, 5)                 5125      
Total params: 25,691,013
Trainable params: 2,103,301
Non-trainable params: 23,587,712
_________________________________________________________________
None
Epoch 1/2
10/10 [==============================] - 1s 56ms/step - loss: 5.4411 - accuracy: 0.3467
Epoch 2/2
5/5 [==============================] - 0s 75ms/step - loss: 3.0097 - accuracy: 0.5682
[3.009725570678711, 0.56818181276